In [1]:
from __future__ import print_function

import ga4gh.client

# GA4GH IPython Example Notebook

This notebook provides an overview of how to call a the GA4GH reference server from an iPython notebook.  Before running this notebook:

1. git clone https://github.com/ga4gh/server.git -b develop
2. Download the example data (if $scripts/download\_data.py$ doens't work, wget https://github.com/ga4gh/server/releases/download/data/ga4gh-example-data-v3.2.tar and tar -xvf in the server/ directory
3. Launch an instance of the reference server on localhost ("python server_dev.py")

Note: If you have trouble importing the `ga4gh` module, either symlink this notebook into the `/server` directory or add the path to the GA4GH development repo to your `PYTHONPATH`.

## Connect to GA4GH Server

In [2]:
baseURL = "http://localhost:8000"
client = ga4gh.client.HttpClient(baseURL)

Great! Now we can run calls to query ReferenceSets, References, Datasets, VariantSets, CallSets, Variants, ReadGroups, ReadGroupSets, & Reads. 

`Search` methods return generators. Here we will explictly create lists out of these objects to allow us to directly index into the list of instances that the query returned.

Instances have a `toJsonDict` method that returns a dictionary representation of the object. We'll make use of this to examine the the underlying data structures.

## Search/Get ReferenceSets

ReferenceSets are lists of References.  Think of them like the genome version (ie. grch37, hg19).

In [3]:
referenceSets = list(client.search_reference_sets())
print("ReferenceSets")
for referenceSet in referenceSets:
    print("NCBI Taxon Id: {}".format(referenceSet.ncbiTaxonId))

ReferenceSets
NCBI Taxon Id: 9606


In [4]:
referenceSet = client.get_reference_set(referenceSets[0].id)
referenceSet.toJsonDict()

{u'assemblyId': u'TODO',
 u'description': u'TODO',
 u'id': u'R1JDaDM4LXN1YnNldA',
 u'isDerived': False,
 u'md5checksum': u'234ea63052f999c21c2bdb6f60e61038',
 u'name': u'GRCh38-subset',
 u'ncbiTaxonId': 9606,
 u'sourceAccessions': [],
 u'sourceURI': u'TODO'}

## Search/Get References 

The References endpoints let you access the DNA sequences that make up the reference genome. The example dataset is from subsets of the first 3 chromosomes:

In [5]:
references = list(client.search_references(referenceSet.id))
print("References")
for reference in references:
    print("Name: {}, Length: {}".format(reference.name, reference.length))

References
Name: 2, Length: 32403
Name: 1, Length: 138395
Name: 3, Length: 81617


In [6]:
reference = client.get_reference(references[0].id)
reference.toJsonDict()

{u'id': u'R1JDaDM4LXN1YnNldDoy',
 u'isDerived': False,
 u'length': 32403,
 u'md5checksum': u'f513b7c19964e17092b55df262f62990',
 u'name': u'2',
 u'ncbiTaxonId': 9606,
 u'sourceAccessions': [u'CM000664.2.subset'],
 u'sourceDivergence': None,
 u'sourceURI': u'http://www.ebi.ac.uk/ena/data/view/CM000664.2%26range=0-32403&display=fasta'}

In addition to fetching metadata about the reference, you can access the base sequence:

In [7]:
client.listReferenceBases(references[0].id, start=10000, end=10100)

u'CGTATCCCACACACCACACCCACACACCACACCCACACACACCCACACCCACACCCACACACACCACACCCACACACCACACCCACACCCACACACCACA'

## Search/Get Dataset

Datasets are a collection of related data. In this example, we'll examine a collection of reads and variants.

In [8]:
datasets = list(client.search_datasets())
print("Datasets")
for dataset in datasets:
    print("Name: {}".format(dataset.name))

Datasets
Name: 1kg-p3-subset


In [9]:
dataset = client.get_dataset(datasets[0].id)
dataset.toJsonDict()

{u'description': None, u'id': u'MWtnLXAzLXN1YnNldA', u'name': u'1kg-p3-subset'}

## Search/Get VariantSets

VariantSets are a collection of variants and calls that are intended to be analyzed together.

In [10]:
variantSets = list(client.search_variant_sets(dataset.id))
print("VariantSets")
for variantSet in variantSets:
    print("Name: {}".format(variantSet.name))

VariantSets
Name: mvncall


In [11]:
variantSetId = variantSets[0].id
variantSet = client.get_variant_set(variantSetId)
variantSet.toJsonDict()

{u'datasetId': u'MWtnLXAzLXN1YnNldA',
 u'id': u'MWtnLXAzLXN1YnNldDptdm5jYWxs',
 u'metadata': [{u'description': u'',
   u'id': u'MWtnLXAzLXN1YnNldDptdm5jYWxsOm1ldGFkYXRhOnZlcnNpb24',
   u'info': {},
   u'key': u'version',
   u'number': u'1',
   u'type': u'String',
   u'value': u'VCFv4.1'},
  {u'description': u'Confidence interval around END for imprecise variants',
   u'id': u'MWtnLXAzLXN1YnNldDptdm5jYWxsOm1ldGFkYXRhOklORk8uQ0lFTkQ',
   u'info': {},
   u'key': u'INFO.CIEND',
   u'number': u'2',
   u'type': u'Integer',
   u'value': u''},
  {u'description': u'Confidence interval around POS for imprecise variants',
   u'id': u'MWtnLXAzLXN1YnNldDptdm5jYWxsOm1ldGFkYXRhOklORk8uQ0lQT1M',
   u'info': {},
   u'key': u'INFO.CIPOS',
   u'number': u'2',
   u'type': u'Integer',
   u'value': u''},
  {u'description': u'Source call set.',
   u'id': u'MWtnLXAzLXN1YnNldDptdm5jYWxsOm1ldGFkYXRhOklORk8uQ1M',
   u'info': {},
   u'key': u'INFO.CS',
   u'number': u'1',
   u'type': u'String',
   u'value': u''},

## Search/Get Callset

Callsets are a collection of genotype calls. Callsets apply to the samples within a dataset.

In [12]:
callSets = list(client.search_call_sets(variantSetId))
print("CallSets")
for callSet in callSets:
    print("Name: {}".format(callSet.name))

CallSets
Name: HG00096
Name: HG00533
Name: HG00534


In [13]:
callSet = client.get_call_set(callSets[0].id)
callSet.toJsonDict()

{u'created': 1453157520705,
 u'id': u'MWtnLXAzLXN1YnNldDptdm5jYWxsOkhHMDAwOTY',
 u'info': {},
 u'name': u'HG00096',
 u'sampleId': u'HG00096',
 u'updated': 1453157520705,
 u'variantSetIds': [u'MWtnLXAzLXN1YnNldDptdm5jYWxs']}

## Search/Get Variants

A Variant represents a change in DNA sequence relative to some reference. For example, a variant could represent a SNP or an insertion. You can think of them as a row in a VCF. Variants can be supported by many calls.

In [14]:
variants = list(client.search_variants(variantSetId, start=100000, end=101000, referenceName = "1"))
print("Variants")
for variant in variants:
    print("Reference Name: {}, Start: {}, Name: {}".format(variant.referenceName, variant.start, variant.names[0]))

Variants
Reference Name: 1, Start: 100648, Name: rs199608293
Reference Name: 1, Start: 100675, Name: rs188226172
Reference Name: 1, Start: 100714, Name: rs561694274
Reference Name: 1, Start: 100857, Name: rs368741663


In [15]:
variant = client.get_variant(variants[0].id)
variant.toJsonDict()

{u'alternateBases': [u'C'],
 u'calls': [{u'callSetId': u'MWtnLXAzLXN1YnNldDptdm5jYWxsOkhHMDAwOTY',
   u'callSetName': u'HG00096',
   u'genotype': [0, 0],
   u'genotypeLikelihood': [],
   u'info': {},
   u'phaseset': u'*'},
  {u'callSetId': u'MWtnLXAzLXN1YnNldDptdm5jYWxsOkhHMDA1MzM',
   u'callSetName': u'HG00533',
   u'genotype': [0, 0],
   u'genotypeLikelihood': [],
   u'info': {},
   u'phaseset': u'*'},
  {u'callSetId': u'MWtnLXAzLXN1YnNldDptdm5jYWxsOkhHMDA1MzQ',
   u'callSetName': u'HG00534',
   u'genotype': [0, 0],
   u'genotypeLikelihood': [],
   u'info': {},
   u'phaseset': u'*'}],
 u'created': 1453157520705,
 u'end': 100649,
 u'id': u'MWtnLXAzLXN1YnNldDptdm5jYWxsOjE6MTAwNjQ4OjNjNjFiODA3MDhlNDIzYTVhMGFiNjlmNGYwZjA2N2Yx',
 u'info': {u'AA': [u'.|||'],
  u'AC': [u'0'],
  u'AF': [u'0.000599041988607'],
  u'AFR_AF': [u'0.0'],
  u'AMR_AF': [u'0.0'],
  u'AN': [u'6'],
  u'DP': [u'16885'],
  u'EAS_AF': [u'0.00300000002608'],
  u'EUR_AF': [u'0.0'],
  u'NS': [u'2504'],
  u'SAS_AF': [u'0.0'],

## Search/Get Readgroupsets

A ReadGroupSet is a logical collection of ReadGroups. Typically one ReadGroupSet represents all the reads from one experimental sample.

In [16]:
readGroupSets = list(client.search_read_group_sets(dataset.id))
print("ReadGroupSets")
for readGroup in readGroupSets:
    print("Name: {}".format(readGroup.name))

ReadGroupSets
Name: HG00533
Name: HG00534


In [17]:
readGroupSet = client.get_read_group_set(readGroupSets[0].id)
readGroupSet.toJsonDict()

{u'datasetId': u'MWtnLXAzLXN1YnNldA',
 u'id': u'MWtnLXAzLXN1YnNldDpIRzAwNTMz',
 u'name': u'HG00533',
 u'readGroups': [{u'created': 1456755492901,
   u'datasetId': u'MWtnLXAzLXN1YnNldA',
   u'description': u'SRP001293',
   u'experiment': {u'description': u'SRP001293',
    u'id': u'MWtnLXAzLXN1YnNldDpIRzAwNTMzOkVSUjAyMDIzNzpleHBlcmltZW50',
    u'info': {},
    u'instrumentDataFile': None,
    u'instrumentModel': u'ILLUMINA',
    u'library': u'HUMwfmREJDIAAPEI-3',
    u'libraryLayout': None,
    u'molecule': None,
    u'name': None,
    u'platformUnit': None,
    u'recordCreateTime': u'2016-02-29T14:18:12Z',
    u'recordUpdateTime': u'2016-02-29T14:18:12Z',
    u'runTime': None,
    u'selection': None,
    u'sequencingCenter': u'BGI',
    u'strategy': None},
   u'id': u'MWtnLXAzLXN1YnNldDpIRzAwNTMzOkVSUjAyMDIzNw',
   u'info': {},
   u'name': u'ERR020237',
   u'predictedInsertSize': 473,
   u'programs': [{u'commandLine': u'bwa index -a bwtsw $reference_fasta',
     u'id': u'bwa_index',
   

## Get ReadGroups

A ReadGroup is a set of reads derived from one physical sequencing process.

In [18]:
readGroup = client.get_read_group(readGroupSet.readGroups[0].id)
readGroup.toJsonDict()

{u'created': 1456755492901,
 u'datasetId': u'MWtnLXAzLXN1YnNldA',
 u'description': u'SRP001293',
 u'experiment': {u'description': u'SRP001293',
  u'id': u'MWtnLXAzLXN1YnNldDpIRzAwNTMzOkVSUjAyMDIzNzpleHBlcmltZW50',
  u'info': {},
  u'instrumentDataFile': None,
  u'instrumentModel': u'ILLUMINA',
  u'library': u'HUMwfmREJDIAAPEI-3',
  u'libraryLayout': None,
  u'molecule': None,
  u'name': None,
  u'platformUnit': None,
  u'recordCreateTime': u'2016-02-29T14:18:12Z',
  u'recordUpdateTime': u'2016-02-29T14:18:12Z',
  u'runTime': None,
  u'selection': None,
  u'sequencingCenter': u'BGI',
  u'strategy': None},
 u'id': u'MWtnLXAzLXN1YnNldDpIRzAwNTMzOkVSUjAyMDIzNw',
 u'info': {},
 u'name': u'ERR020237',
 u'predictedInsertSize': 473,
 u'programs': [{u'commandLine': u'bwa index -a bwtsw $reference_fasta',
   u'id': u'bwa_index',
   u'name': u'bwa',
   u'prevProgramId': None,
   u'version': u'0.5.9-r16'},
  {u'commandLine': u'bwa aln -q 15 -f $sai_file $reference_fasta $fastq_file\tPP:bwa_index',

## Get Reads

Each read alignment describes an alignment with additional information about the fragment and the read. A read alignment object is equivalent to a line in a SAM file.

Since there are often many reads, we won't iterate over all of them. Instead we can summerize the query and then output the dictionary representing the last one.

In [19]:
readGroupIds = [readGroup.id for readGroup in readGroupSet.readGroups]
readGroupDescriptions = [readGroup.description for readGroup in readGroupSet.readGroups]
reads = client.search_reads(readGroupIds, reference.id)
print("Read Alignments")
# The server paginates reads by default; here we iterate over the reads 
# generate to make the appropriate requests to the server
count = 0
for read in reads:
    count += 1
print("{} reads in Readgroups: {} on Reference: {}".format(
      count, ', '.join(readGroupDescriptions), reference.name))

Read Alignments
984 reads in Readgroups: SRP001293 on Reference: 2


In [20]:
read.toJsonDict()

{u'alignedQuality': [31,
  31,
  33,
  27,
  36,
  36,
  38,
  37,
  37,
  35,
  35,
  37,
  37,
  30,
  35,
  39,
  39,
  40,
  39,
  39,
  41,
  37,
  41,
  40,
  40,
  40,
  40,
  36,
  39,
  33,
  40,
  39,
  39,
  38,
  29,
  42,
  38,
  37,
  36,
  29,
  31,
  32,
  34,
  30,
  34,
  34,
  39,
  34,
  36,
  39,
  16,
  35,
  35,
  33,
  32,
  39,
  35,
  35,
  29,
  27,
  41,
  37,
  37,
  40,
  39,
  41,
  31,
  31,
  38,
  30,
  38,
  31,
  38,
  34,
  31,
  23,
  16,
  15,
  27,
  25,
  36,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2],
 u'alignedSequence': u'GGTTCCAAGGTGGGTGCATAACTGAGGCTCATGCCAGGTGCCACACAGCACTCCCTTGGCCCCCTAATGGACTTCGGGACAGGCAAGGGCC',
 u'alignment': {u'cigar': [{u'operation': u'ALIGNMENT_MATCH',
    u'operationLength': 82,
    u'referenceSequence': None},
   {u'operation': u'CLIP_SOFT',
    u'operationLength': 9,
    u'referenceSequence': None}],
  u'mappingQuality': 60,
  u'position': {u'position': 27497,
   u'referenceName': u'2',
   u'strand': u'POS_STR